In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
import gc

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training variables
BATCH_SIZE = 32
EPOCHS = 300
LEARNING_RATE = 0.00005

# Allowed transformations for the train, validation, and test datasets
train_transform = transforms.Compose([
    transforms.Resize(256, antialias=True),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224), antialias=True),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the train, validation, and test datasets
train_dataset = datasets.Flowers102(root="./data", split="train", transform=train_transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_dataset = datasets.Flowers102(root="./data", split="val", transform=val_test_transform, download=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_dataset = datasets.Flowers102(root="./data", split="test", transform=val_test_transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 224 -> 112
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 112 -> 56
        self.conv3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 56 -> 28
        self.conv4 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(256),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 28 -> 14
        self.conv5 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(512),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 14 -> 7
        self.conv6 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(512),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 7 -> 3
        self.conv7 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(512),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 3 -> 1
        self.conv8 = nn.Sequential(nn.Conv2d(512, 1024, kernel_size=1, stride=1, padding=0),
                                   nn.BatchNorm2d(1024),
                                   nn.ReLU()) # Removed pooling layer
        self.conv9 = nn.Sequential(nn.Conv2d(1024, 1024, kernel_size=1, stride=1, padding=0),
                                   nn.BatchNorm2d(1024),
                                   nn.ReLU())

        self.fc1 = nn.Sequential(nn.Linear(1 * 1024 * 1, 4096),
                                   nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(4096, 4096),
                                   nn.ReLU())
        self.fc3 = nn.Sequential(nn.Linear(4096, 102))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = x.view(-1, 1024 * 1 * 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

# Clear CUDA cache
gc.collect()
torch.cuda.empty_cache()

# Create an instance of the CNN and move it to the device
cnn = CNN().to(device)

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=LEARNING_RATE, weight_decay=0.0001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)


# Train the CNN
for epoch in range(EPOCHS):
    cnn.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 16 == 0:
            print(f"Epoch [{epoch + 1}/{EPOCHS}], Step [{i + 1}/{len(train_loader)}], Step Loss: {loss.item():.4f}")

    # Evaluate model after each training epoch
    cnn.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = cnn(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{EPOCHS}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%')

    # Update the learning rate scheduler
    scheduler.step(val_loss)

# Test the CNN on the test set
cnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

100%|██████████| 344862509/344862509 [00:24<00:00, 14050727.66it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 572936.22it/s]


100%|██████████| 14989/14989 [00:00<00:00, 41552163.02it/s]


Epoch [1/300], Step [16/32], Step Loss: 4.6306
Epoch [1/300], Step [32/32], Step Loss: 4.4927
Epoch [1/300], Validation Loss: 4.5142, Validation Accuracy: 5.88%
Epoch [2/300], Step [16/32], Step Loss: 4.2696
Epoch [2/300], Step [32/32], Step Loss: 3.7755
Epoch [2/300], Validation Loss: 3.8367, Validation Accuracy: 13.14%
Epoch [3/300], Step [16/32], Step Loss: 3.6046
Epoch [3/300], Step [32/32], Step Loss: 3.7423
Epoch [3/300], Validation Loss: 3.5205, Validation Accuracy: 15.10%
Epoch [4/300], Step [16/32], Step Loss: 2.9442
Epoch [4/300], Step [32/32], Step Loss: 3.1773
Epoch [4/300], Validation Loss: 3.2510, Validation Accuracy: 19.51%
Epoch [5/300], Step [16/32], Step Loss: 2.8350
Epoch [5/300], Step [32/32], Step Loss: 2.8396
Epoch [5/300], Validation Loss: 3.0203, Validation Accuracy: 22.55%
Epoch [6/300], Step [16/32], Step Loss: 2.5587
Epoch [6/300], Step [32/32], Step Loss: 2.6363
Epoch [6/300], Validation Loss: 3.3962, Validation Accuracy: 19.51%
Epoch [7/300], Step [16/32], 